## 가중치 초기화(xavier)

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda'

random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# 파라미터
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# 데이터 가져오기
mnist_train = dsets.MNIST(root='./MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='./MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
# 배치 사이즈만큼 이미지 데이터들이 묶여있음
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle=True,
                                         drop_last = True)

In [8]:
# nn layers
linear1 = torch.nn.Linear(784,512, bias = True)
linear2 = torch.nn.Linear(512,512, bias = True)
linear3 = torch.nn.Linear(512,512, bias = True)
linear4 = torch.nn.Linear(512,512, bias = True)
linear5 = torch.nn.Linear(512,10, bias = True)

relu = torch.nn.ReLU()

In [9]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0740,  0.0187, -0.0574,  ..., -0.0222, -0.0648,  0.0520],
        [-0.0291,  0.0522, -0.0714,  ..., -0.0521, -0.0983, -0.0065],
        [ 0.0889, -0.0081, -0.0600,  ..., -0.0100,  0.0868,  0.0456],
        ...,
        [-0.0461, -0.0889, -0.0729,  ..., -0.0090,  0.0636, -0.0572],
        [ 0.0822,  0.0670,  0.0546,  ..., -0.0046, -0.0461, -0.1039],
        [ 0.0055, -0.0434, -0.0374,  ..., -0.0704, -0.0504, -0.0068]],
       requires_grad=True)

In [13]:
# 모델 생성
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4, relu, linear5).to(device)

# cost, loss, optimizer 정의
criterion = torch.nn.CrossEntropyLoss().to(device) # 소프트맥스(출력층 활성화함수) 함수가 내장됨
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [14]:
# training
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.view(-1,28*28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch # 배치묶음별 평균 cost 저장
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.215799943
Epoch: 0002 cost = 0.091863021
Epoch: 0003 cost = 0.067131549
Epoch: 0004 cost = 0.051596958
Epoch: 0005 cost = 0.040742286
Epoch: 0006 cost = 0.032920625
Epoch: 0007 cost = 0.029942259
Epoch: 0008 cost = 0.028333234
Epoch: 0009 cost = 0.023690661
Epoch: 0010 cost = 0.022386836
Epoch: 0011 cost = 0.019931471
Epoch: 0012 cost = 0.018685168
Epoch: 0013 cost = 0.019478919
Epoch: 0014 cost = 0.017803261
Epoch: 0015 cost = 0.013195137
Learning finished


In [15]:
# test
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    # 이미지 하나 예측
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28*28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print(Y_single_data)
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9793999791145325
tensor([9], device='cuda:0')
Label:  9
Prediction:  9


/home/ubuntu/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/ubuntu/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
